In [46]:
from __future__ import division
import numpy as np
import sys

In [47]:
X_train = np.genfromtxt('X_train_C.csv', delimiter=',')
y_train = np.genfromtxt('Y_train_C.csv', delimiter=',')
X_test = np.genfromtxt('X_test_C.csv', delimiter=',')

In [48]:
# X_train = np.array([[1, 1, 2, 3], [2, 7, -4, 8], [6, 5, 4, 2], [4, -8, -1, 0], [-1, -3, 1, 4], [-8, -2, 3, 1], [4, -1, 2, 7]])
# y_train = np.array([0., 1., 1., 0., 1., 1., 0.])

# X_test = np.array([[2, 4, 7, 1], [-1, 2, -4, 3], [4, -1, 3, 9], [-2, 4, -1, 6], [3, -3, 4, 2]])

In [49]:
Y, n_y = np.unique(y_train, return_counts=True)

In [50]:
print(Y, n_y)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.] [41 47 42 37 35 34 42 23 32 17]


In [51]:
pi_hat = []
mu_hat = []
sigma_hat = []
for y, count_y in zip(Y, n_y):
    
    pi_hat_y = count_y/np.sum(n_y)
    pi_hat.append(pi_hat_y)
    mu_hat_y = np.dot((y_train == y), X_train)/count_y
    mu_hat.append(mu_hat_y)
    X_train_y = X_train[y_train.astype(int) == np.int(y), :]
    
    x_mu_hat_dev_y = X_train_y - mu_hat_y
        
    sigma_hat_y = np.dot(x_mu_hat_dev_y.T, x_mu_hat_dev_y)/(count_y)
#     print("*****************************************")
#     print("sigma_hat determinent", np.linalg.det(sigma_hat_y))
    
    sigma_hat.append(sigma_hat_y)

In [52]:
# predicting for X_test
y_test = []
f_x_hat = []
    
for x in X_test:
    print("x test - ", x)
    
    for y in Y:
        idx = np.int(y)
        x_mu_hat_dev_y = x - mu_hat[idx]

        coefficient = pi_hat[idx]/np.sqrt(np.linalg.det(sigma_hat[idx]))
        
        power_value = -(1/2)*(x_mu_hat_dev_y.T@np.linalg.inv(sigma_hat[idx])@x_mu_hat_dev_y)
        
        print("y - ", y)
        
        print("****************")

        print("pi_hat[idx] - ", pi_hat[idx])
        print("coefficient - ", coefficient)
        
        print("power_value - ", power_value)
        print("exponent - ", np.exp(power_value))

        print("pdf value - ", coefficient*np.exp(power_value))

        f_x_hat_y = coefficient*np.exp(power_value)
        f_x_hat.append(f_x_hat_y)

    y_test.append(f_x_hat)


x test -  [0.30957  0.34964  0.013785 0.52674  0.49248  0.277   ]
y -  0.0
****************
pi_hat[idx] -  0.11714285714285715
coefficient -  4.559468770315837
power_value -  -1.2129703748210403
exponent -  0.29731283595397817
pdf value -  1.3555885905461988
y -  1.0
****************
pi_hat[idx] -  0.13428571428571429
coefficient -  9.126242944813114
power_value -  -1.0667099938957763
exponent -  0.34413887595846643
pdf value -  3.1406949887518696
y -  2.0
****************
pi_hat[idx] -  0.12
coefficient -  15.778695271858615
power_value -  -2.1539428654861847
exponent -  0.1160257806616062
pdf value -  1.8307354367389903
y -  3.0
****************
pi_hat[idx] -  0.10571428571428572
coefficient -  5.178618148701072
power_value -  -0.8090483320232738
exponent -  0.44528162491040635
pdf value -  2.3059435040441336
y -  4.0
****************
pi_hat[idx] -  0.1
coefficient -  8.54697357546523
power_value -  -1.4946625281694474
exponent -  0.2243242950886361
pdf value -  1.9172938224574376
y 

pi_hat[idx] -  0.06571428571428571
coefficient -  4.469029111118441
power_value -  -7.696510149325116
exponent -  0.0004544102428513621
pdf value -  0.0020307726036931377
y -  8.0
****************
pi_hat[idx] -  0.09142857142857143
coefficient -  8.560468475650062
power_value -  -15.529772296528993
exponent -  1.80096630042744e-07
pdf value -  1.5417115240517218e-06
y -  9.0
****************
pi_hat[idx] -  0.04857142857142857
coefficient -  12.362837047171496
power_value -  -30.459143156391676
exponent -  5.912378154722636e-14
pdf value -  7.309376768809245e-13
x test -  [ 1.4821   2.4902   3.1314   2.3233  -1.6461  -0.80885]
y -  0.0
****************
pi_hat[idx] -  0.11714285714285715
coefficient -  4.559468770315837
power_value -  -8.098419140512425
exponent -  0.0003040193702949636
pdf value -  0.0013861668244309726
y -  1.0
****************
pi_hat[idx] -  0.13428571428571429
coefficient -  9.126242944813114
power_value -  -11.30772978053915
exponent -  1.227765296011121e-05
pdf val

pdf value -  1.5523576361263536e-05
y -  1.0
****************
pi_hat[idx] -  0.13428571428571429
coefficient -  9.126242944813114
power_value -  -6.80032501239139
exponent -  0.0011134132159398942
pdf value -  0.01016127950663314
y -  2.0
****************
pi_hat[idx] -  0.12
coefficient -  15.778695271858615
power_value -  -4.962222906794558
exponent -  0.006997356061366731
pdf value -  0.11040914900099845
y -  3.0
****************
pi_hat[idx] -  0.10571428571428572
coefficient -  5.178618148701072
power_value -  -6.297524880372103
exponent -  0.0018408554804698188
pdf value -  0.009533087600296836
y -  4.0
****************
pi_hat[idx] -  0.1
coefficient -  8.54697357546523
power_value -  -10.936303517240654
exponent -  1.780015278563104e-05
pdf value -  0.0001521374354980323
y -  5.0
****************
pi_hat[idx] -  0.09714285714285714
coefficient -  6.623103742879091
power_value -  -6.606947440713778
exponent -  0.0013509497157570046
pdf value -  0.008947480118871661
y -  6.0
********

coefficient -  8.54697357546523
power_value -  -2.05988390737225
exponent -  0.12746876722001657
pdf value -  1.0894721851266103
y -  5.0
****************
pi_hat[idx] -  0.09714285714285714
coefficient -  6.623103742879091
power_value -  -3.427988021407722
exponent -  0.032452168200501785
pdf value -  0.21493407667328518
y -  6.0
****************
pi_hat[idx] -  0.12
coefficient -  8.793651106893547
power_value -  -2.5173858309881645
exponent -  0.08067021692555904
pdf value -  0.7093857423607848
y -  7.0
****************
pi_hat[idx] -  0.06571428571428571
coefficient -  4.469029111118441
power_value -  -2.8285788791807347
exponent -  0.05909677768335916
pdf value -  0.26410521984022667
y -  8.0
****************
pi_hat[idx] -  0.09142857142857143
coefficient -  8.560468475650062
power_value -  -1.4327033453228004
exponent -  0.23866286124454789
pdf value -  2.043065899992397
y -  9.0
****************
pi_hat[idx] -  0.04857142857142857
coefficient -  12.362837047171496
power_value -  -10

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
qda = QuadraticDiscriminantAnalysis()
model = 